In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import json

In [2]:
def json_reader(begin):
    if begin not in np.arange(0, 11000, 1000):
        raise Exception(
            "Invalid start pid! Start pids must be {0, 1000, 2000, ..., 11000}"
        )

    end = begin + 999
    path = "../data/playlists/playlist.slice." + str(begin) + "-" + str(end) + ".json"

    jsonData = json.load(open(path, "r"))
    actualSlice = pd.DataFrame.from_dict(jsonData["playlists"], orient="columns")
    return actualSlice

In [3]:
jsonList = []
for begin in np.arange(0, 11000, 1000):
    actual = json_reader(begin)
    jsonList.append(actual)

trainData = pd.concat(jsonList, ignore_index=True)
jsonList.clear()

print(trainData.shape)
trainData.head()

(11000, 12)


,collaborative,description,duration_ms,modified_at,name,num_albums,num_artists,num_edits,num_followers,num_tracks,pid,tracks
0,false,NaN,11532414,1493424000,Throwbacks,47,37,6,1,52,0,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't..."
1,false,NaN,11656470,1506556800,Awesome Playlist,23,21,5,1,39,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_..."
2,false,NaN,14039958,1505692800,korean,51,31,18,1,64,2,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri..."
3,false,NaN,28926058,1501027200,mat,107,86,4,1,126,3,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën..."
4,false,NaN,4335282,1401667200,90s,16,16,7,2,17,4,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk..."


In [4]:
# Turn playlist level dataframe into song level dataframe
# is a df of all track ids, cooresponding artist names, track names and playlist ids

songPlaylistArray = []
for index, row in trainData.iterrows():
    for track in row["tracks"]:
        songPlaylistArray.append(
            [track["track_uri"], track["artist_name"], track["track_name"], row["pid"]]
        )
songPlaylist = pd.DataFrame(
    songPlaylistArray, columns=["trackid", "artist_name", "track_name", "pid"]
)
songPlaylistArray.clear()

print(songPlaylist.shape)
songPlaylist.head(10)

(731360, 4)


,trackid,artist_name,track_name,pid
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,0
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,0
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,0
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,0
5,spotify:track:0XUfyU2QviPAs6bxSpXYG4,Usher,Yeah!,0
6,spotify:track:68vgtRHr7iZHpzGpon6Jlo,Usher,My Boo,0
7,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,The Pussycat Dolls,Buttons,0
8,spotify:track:7H6ev70Weq6DdpZyyTmUXk,Destiny's Child,Say My Name,0
9,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,OutKast,Hey Ya! - Radio Mix / Club Mix,0


In [5]:
# Turn songs into their unqiue cat codes so we have a 0-N index for tracks
songPlaylist["trackindex"] = songPlaylist["trackid"].astype("category").cat.codes

print(len(songPlaylist["trackid"].unique()))
print(songPlaylist.shape)
songPlaylist.head(10)

180409
(731360, 5)


,trackid,artist_name,track_name,pid,trackindex
0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),0,11647
1,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney Spears,Toxic,0,145808
2,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,Beyoncé,Crazy In Love,0,12511
3,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin Timberlake,Rock Your Body,0,27379
4,spotify:track:1lzr43nnXAijIGYnCT8M8H,Shaggy,It Wasn't Me,0,41255
5,spotify:track:0XUfyU2QviPAs6bxSpXYG4,Usher,Yeah!,0,12745
6,spotify:track:68vgtRHr7iZHpzGpon6Jlo,Usher,My Boo,0,142288
7,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,The Pussycat Dolls,Buttons,0,73997
8,spotify:track:7H6ev70Weq6DdpZyyTmUXk,Destiny's Child,Say My Name,0,168512
9,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,OutKast,Hey Ya! - Radio Mix / Club Mix,0,56109


In [6]:
# Save data in DOK (Dictionary Of Keys) matrix (optimized sparse matrix object)
# Create a sparse pid x trackindex matrix
# If a pid i has song j, mat[i,j]=1

mat = sp.dok_matrix((11000, 180409), dtype=np.float32)
for pid, trackindex in zip(songPlaylist["pid"], songPlaylist["trackindex"]):
    mat[pid, trackindex] = 1.0

In [7]:
import keras
from keras import backend as K
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, Reshape, multiply, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from time import time

Using TensorFlow backend.


In [8]:
def get_model(num_playlists, num_items, latent_dim, regs=[0, 0]):
    # Input variables
    playlist_input = Input(shape=(1,), dtype="int32", name="playlist_input")

    music_input = Input(shape=(1,), dtype="int32", name="music_input")

    playlist_embedding = Embedding(
        name="playlist_embedding",
        input_dim=num_playlists,
        output_dim=latent_dim,
        embeddings_regularizer=l2(regs[0]),
        input_length=1,
    )

    music_embedding = Embedding(
        name="music_embedding",
        input_dim=num_items,
        output_dim=latent_dim,
        embeddings_regularizer=l2(regs[1]),
        input_length=1,
    )

    # Flattens the embedding vector
    playlist_latent = Flatten()(playlist_embedding(playlist_input))
    music_latent = Flatten()(music_embedding(music_input))

    # Element-wise product of playlist and music embeddings
    predict_vector = multiply([playlist_latent, music_latent])

    # Final prediction layer
    prediction = Dense(
        1,
        activation="sigmoid",
        kernel_initializer="random_normal",
        name="prediction_layer",
    )(predict_vector)

    model = Model(inputs=[playlist_input, music_input], outputs=prediction)

    return model

In [9]:
def get_train_instances(train, num_negatives):
    playlist_input, music_input, labels = [], [], []
    num_playlists, num_musics = train.shape

    for (u, i) in train.keys():
        # positive instances
        playlist_input.append(u)
        music_input.append(i)
        labels.append(1)

        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_musics)

            while (u, j) in train:
                j = np.random.randint(num_musics)

            playlist_input.append(u)
            music_input.append(j)
            labels.append(0)

    return playlist_input, music_input, labels

In [10]:
# Specify hyperparameters
num_factors = 8
regs = [0, 0]
num_negatives = 4
learning_rate = 0.001
epochs = 30
batch_size = 1000

# Loading data
train = mat
num_playlists, num_musics = train.shape

# Build and compile model
model = get_model(num_playlists, num_musics, num_factors, regs)
model.compile(
    optimizer=Adam(lr=learning_rate), loss="binary_crossentropy", metrics=["acc"]
)
print(model.summary())

history = []
# Train model
for epoch in range(epochs):
    print("Training Epoch: %d/%d" % (epoch + 1, epochs))

    # Generate training instances
    playlist_input, music_input, labels = get_train_instances(train, num_negatives)

    # Train model
    hist = model.fit(
        [np.array(playlist_input), np.array(music_input)],  # input
        np.array(labels),  # labels
        validation_split=0.2,
        batch_size=batch_size,
        shuffle=True,
    )

    history.append(hist.history)

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
playlist_input (InputLayer)     (None, 1)            0                                            
__________________________________________________________________________________________________
music_input (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
playlist_embedding (Embedding)  (None, 1, 8)         88000       playlist_input[0][0]             
__________________________________________________________________________________________________
music_embedding (Embedding)     (None, 1, 8)         1443272     music_input[0][0]                
_____________________________________

2888180/2888180 [==============================] - 17s 6us/step - loss: 0.0835 - acc: 0.9715 - val_loss: 1.6501 - val_acc: 0.8000
Training Epoch: 27/30
Train on 2888180 samples, validate on 722045 samples
Epoch 1/1
2888180/2888180 [==============================] - 17s 6us/step - loss: 0.0825 - acc: 0.9719 - val_loss: 1.6964 - val_acc: 0.8000
Training Epoch: 28/30
Train on 2888180 samples, validate on 722045 samples
Epoch 1/1
2888180/2888180 [==============================] - 17s 6us/step - loss: 0.0824 - acc: 0.9718 - val_loss: 1.7390 - val_acc: 0.8000
Training Epoch: 29/30
Train on 2888180 samples, validate on 722045 samples
Epoch 1/1
2888180/2888180 [==============================] - 18s 6us/step - loss: 0.0813 - acc: 0.9725 - val_loss: 1.7774 - val_acc: 0.8000
Training Epoch: 30/30
Train on 2888180 samples, validate on 722045 samples
Epoch 1/1
2888180/2888180 [==============================] - 17s 6us/step - loss: 0.0800 - acc: 0.9730 - val_loss: 1.8219 - val_acc: 0.8000


In [11]:
history_df = pd.DataFrame(history)
history_df.set_index(pd.Series(range(1, epochs+1)), inplace=True)

history_df.index.name = "Epoch"
history_df.columns = [
    "Training Accuracy",
    "Training Loss",
    "Validation Accuracy",
    "Validation Loss",
]

pd.options.display.float_format = '{:,.20f}'.format

for column in history_df.columns:

    history_df.loc[:, column] = history_df[column].map(
        lambda x: x[0]
    )

display(history_df)

,Training Accuracy,Training Loss,Validation Accuracy,Validation Loss
Epoch,,,,
1,0.80425112013017452384,0.52267931353716579057,0.80000001192092895508,0.50043660887236562917
2,0.86814637566191632523,0.34062294104612417245,0.80001247652263363097,0.50080104500643685128
3,0.88518409564804167999,0.28626433539298024034,0.80001109158523309173,0.50164317116469969715
4,0.89599990274266416712,0.25074461957752813834,0.80000555175308096789,0.50608306453354778842
5,0.90772320231692060766,0.21982862348095363880,0.80001663133483535972,0.51698203382864105215
6,0.91987930235037795867,0.19349803678423976860,0.80001801635478564378,0.54172474329562503303
7,0.92968097696259155427,0.17323478952252766017,0.80001524639743482048,0.57566004452027386140
8,0.93695995407099741392,0.15803340217569708526,0.80001386146003417021,0.62433422689431794517
9,0.94257975554391792983,0.14581911976066966319,0.80001663133483535972,0.67332403701706311505
